Use this code to generate synthetic data for Lutein production. The inputs of the data-driven model would be concentration of biomass (C_X) in the reactor, concentration of nitrate (C_N) in the reactor, concentration of Lutein (C_L) in the reactor, influx of nitrate (F_in), inlet nitrate concentration (C_N_in), light intensity (I0), and time t hours. The output should be C_X, C_N, and C_L at t+1 hours. 

In [1]:
# Following are parameters of the model mostly taken from https://aiche.onlinelibrary.wiley.com/doi/epdf/10.1002/aic.15667 but changed slightly (after unit conversion)
# Do not change this... 

u_m = 0.152 # 1/h
u_d = 5.95*1e-3 # 1/h
K_N = 30.0*1e-3 # g/L

Y_NX = 0.305 # g/g

k_m = 0.350*1e-3*2 #g/g-h

k_d = 3.71*0.05/90 # L/g-h

K_NL = 10.0*1e-3 # g/L

k_s = 142.8 # umol/m2-s

k_i = 214.2 # umol/m2-s

k_sL = 320.6 # umol/m2-s

k_iL = 480.9 # umol/m2-s

tau = 0.120 #m2/g

Ka = 0.0 #1/m

In [2]:
import numpy as np
import random

#To generate data, just change these values in this block (perhaps in a loop), In my opinion, 
#a good range for C_x0 (which is the initial concnentration of biomass in the reactor C_X) is 0.2 - 2 g/L
# a good range for C_N0 (which is the initial concnetraiton of nitrate in the reactor C_N) is 0.2 - 2 g/L
# a good range for F_in (the inlet flow rate of nitrate into the reactor) is 1e-3 1.5e-2 L/h
# a good range for C_N_in (the inlet concentration of nitrate feed to the reactor) is 5 - 15 g/L
# a good range for intensity of light is 100 - 200 umol/m2-s

C_x0 = 0.5 # g/L

C_N0 = 1 #g/L

F_in = 8e-3 #L/h

C_N_in = 10 #g/L

I0 = 150 # umol/m2-s

C_x0_r = (0.2, 2)
C_N0_r = (0.2, 2)
F_in_r = (1e-3, 1.5e-2)
C_N_in_r = (5, 15)
I0_r = (100, 200)

num_values = 100

synth_data = []

for x in range(num_values):
    C_x0 = round(random.uniform(C_x0_r[0], C_x0_r[1]), 1)
    C_N0 = round(random.uniform(C_N0_r[0], C_N0_r[1]), 1)
    F_in = round(random.uniform(F_in_r[0], F_in_r[1]), 3)
    C_N_in = round(random.uniform(C_N_in_r[0], C_N_in_r[1]), 0)
    I0 = round(random.uniform(I0_r[0], I0_r[1]), 0)
    
    synth_data.append((C_x0, C_N0, F_in, C_N_in, I0))

#for i, value in enumerate(synth_data):
 #   print(f"Value {i+1}: {value}")


The code below formulates and solves the ODE model in the paper https://aiche.onlinelibrary.wiley.com/doi/epdf/10.1002/aic.15667

We specifically use equations 1, 2, and 3. We approximate 3 for some intermediate value of Z to avoid using the averaging shown in equation 4. 

In [3]:
    
    
def pbr(t,C): # returns the RHS of the ODE model
    C_X = C[0] # concentration of biomass
    C_N = C[1] # concentration of nitrate
    C_L = C[2] # concentration of lutein
    
    I = 2*I0*(np.exp(-(tau*0.01*1000*C_X))) # computing attenuated intensity within the reactor. 
    
    Iscaling_u = I/(I+k_s + I**2/k_i)
    Iscaling_k = I/(I + k_sL + I**2/k_iL)
    
    u0 = u_m*Iscaling_u
    k0 = k_m*Iscaling_k
    
        
    #print(u_m, k_m, u0, k0,u_d)
    

    
    dCxdt = u0*C_N*C_X/(C_N + K_N) - u_d*C_X
    
    #print(C_N, C_X, (C_N + K_N), dCxdt, C_N*C_X/(C_N + K_N), u_d*C_X, C_N/(C_N + K_N))
    
    dCndt = -Y_NX*u0*C_N*C_X/(C_N + K_N) + F_in * C_N_in
    
    dCldt = k0*C_N*C_X/(C_N + K_NL) - k_d*C_L*C_X
    
    #print(k0, k_d, k0/k_d )
    
    return np.array([dCxdt, dCndt, dCldt])

In [20]:
from scipy.integrate import solve_ivp
import matplotlib.pyplot as plt
import pandas as pd

mainData = pd.DataFrame()
for i, data in enumerate(synth_data):
    C_x0, C_N0, F_in, C_N_in, I0 = data
    
    ta = np.linspace(0,150,200)

    y = solve_ivp(pbr, [0, 150], np.array([C_x0, C_N0, 0.0]), t_eval=ta) # solves the ODE model for a given set of initial conditions and simulation time

    t = y.t
    C = y.y
    #print(C[2]*1000)
    save = pd.DataFrame()
    save['Time'] = t
    save['C_X'] = C[0]
    save['C_N'] = C[1]
    save['C_L'] = C[2]
    save['C_x0'] = C_x0
    save['C_N0'] = C_N0
    save['F_in'] = F_in
    save['C_N_in'] = C_N_in
    save['I0'] = I0
    mainData = pd.concat([mainData, save])
from pathlib import Path  

filepath = Path('Documents\STEMVisualsSynthData.csv')  

filepath.parent.mkdir(parents=True, exist_ok=True)  

mainData.to_csv(filepath) 



###### 